In [33]:
import pandas as pd
import json
import re

def to_number(string):
    string = string.replace(",", "")
    return float(string)

separate_origins_df = pd.read_csv("separate_origins.csv", index_col=0)
separate_origins_df = separate_origins_df[pd.notna(separate_origins_df.country_destination)]
separate_origins_df["price"] = separate_origins_df["price"].apply(to_number)
print(separate_origins_df.columns)

Index(['origin', 'country_origin', 'area_level_1_origin', 'clean_origin',
       'origin_lat', 'origin_long', 'destination', 'country_destination',
       'area_level_1_destination', 'clean_destination', 'destination_lat',
       'destination_long', 'date', 'price', 'url', 'note'],
      dtype='object')


In [34]:
countries = set(separate_origins_df.country_origin.unique())
countries.update(separate_origins_df.country_destination.unique())

props = []
for c in countries:
    props.append({"name":c})
params = {
    "props": props
}
fields = re.sub(r'(?<!: )"(\S*?)"', '\\1', json.dumps(props))
print(":params props =>", fields)

:params props => [{name: "Belgium (BE)"}, {name: "Finland (FI)"}, {name: "Spain (ES)"}, {name: "Ecuador (EC)"}, {name: "Paraguay (PY)"}, {name: "Indonesia (ID)"}, {name: "India (IN)"}, {name: "Singapore (SG)"}, {name: "Turkey (TR)"}, {name: "Malta (MT)"}, {name: "Vietnam (VN)"}, {name: "South Africa (ZA)"}, {name: "United States (US)"}, {name: "Iceland (IS)"}, {name: "Aruba (AW)"}, {name: "Mongolia (MN)"}, {name: "Brazil (BR)"}, {name: "Hong Kong (HK)"}, {name: "Germany (DE)"}, {name: "Oman (OM)"}, {name: "Sint Maarten (SX)"}, {name: "Argentina (AR)"}, {name: "Australia (AU)"}, {name: "Venezuela (VE)"}, {name: "Ireland (IE)"}, {name: "Japan (JP)"}, {name: "Mexico (MX)"}, {name: "Canada (CA)"}, {name: "Switzerland (CH)"}, {name: "France (FR)"}, {name: "Romania (RO)"}, {name: "Puerto Rico (PR)"}, {name: "Macau (MO)"}, {name: "Cuba (CU)"}, {name: "U.S. Virgin Islands (VI)"}, {name: "The Bahamas (BS)"}, {name: "Lebanon (LB)"}, {name: "United Kingdom (GB)"}, {name: "Trinidad and Tobago (TT)

```
UNWIND $props AS map
CREATE (n:Country)
SET n = map
```

```
CREATE INDEX ON :Country(name)
```


In [96]:
origins = separate_origins_df[["country_origin","area_level_1_origin"]]
origins.columns = ["country", "city"]

destinations = separate_origins_df[["country_destination","area_level_1_destination"]]
destinations.columns = ["country", "city"]

joint_cities = pd.concat ([origins,destinations])
joint_cities = joint_cities[pd.notna(joint_cities.city)]
    

In [127]:
ex = separate_origins_df[["country_origin","area_level_1_origin", 
                          "country_destination","area_level_1_destination","price","date","url" ]]
dictionary = ex.to_dict("record")
fields = re.sub(r'(?<!: )"(\S*?)"', '\\1', json.dumps(dictionary))

:params props => [{country_origin: "Mexico (MX)", area_level_1_origin: "Mexico City (CDMX)", country_destination: "Norway (NO)", area_level_1_destination: "Oslo (Oslo)", price: 14922.0, date: "2018-06-19", url: "https://www.vuelax.com/2018/06/19/oslo/"}, {country_origin: "Mexico (MX)", area_level_1_origin: "Mexico City (CDMX)", country_destination: "Finland (FI)", area_level_1_destination: NaN, price: 15478.0, date: "2018-06-19", url: "https://www.vuelax.com/2018/06/19/helsinki/"}, {country_origin: "Mexico (MX)", area_level_1_origin: "Mexico City (CDMX)", country_destination: "Thailand (TH)", area_level_1_destination: NaN, price: 13300.0, date: "2018-06-18", url: "https://www.vuelax.com/2018/06/18/tai-4/"}, {country_origin: "Mexico (MX)", area_level_1_origin: "Jalisco (Jal.)", country_destination: "Thailand (TH)", area_level_1_destination: NaN, price: 13300.0, date: "2018-06-18", url: "https://www.vuelax.com/2018/06/18/tai-4/"}, {country_origin: "Mexico (MX)", area_level_1_origin: "Mex

In [128]:
from neo4j.v1 import GraphDatabase

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "g1"))

In [129]:
with driver.session() as s:
    result = s.run("""UNWIND $props AS map
MATCH (cityFrom:City{name:map.area_level_1_origin})
MATCH (cityTo:City{name:map.area_level_1_destination})
MERGE (cityFrom)-[:Flight{price:map.price,date:date(map.date)}]->(cityTo)
""", parameters={"props":dictionary})
    print(result)
    